In [ ]:
# program to score average sentiment with weight added for # of likes and # of retweets using polyglot library for polarity scoring

In [ ]:
import datetime
import pandas as pd
import numpy as np
import warnings
import csv

In [ ]:
!pip install polyglot
!pip install pyicu  
!pip install Morfessor   
!pip install pycld2      

In [ ]:
%%bash
polyglot download ner2.en    
polyglot download pos2.en    
polyglot download sentiment2.en  
polyglot download embeddings2.en

In [ ]:
from polyglot.detect import Detector
from polyglot.text import Text 
from polyglot.downloader import downloader

In [ ]:
# downloading sentiment packages for a bunch of different languages
!polyglot download sentiment2.af
!polyglot download sentiment2.als
!polyglot download sentiment2.am
!polyglot download sentiment2.an
!polyglot download sentiment2.ar
!polyglot download sentiment2.arz
!polyglot download sentiment2.as
!polyglot download sentiment2.ast
!polyglot download sentiment2.az
!polyglot download sentiment2.ba
!polyglot download sentiment2.bar
!polyglot download sentiment2.be
!polyglot download sentiment2.bg
!polyglot download sentiment2.bn
!polyglot download sentiment2.bo
!polyglot download sentiment2.bpy
!polyglot download sentiment2.br
!polyglot download sentiment2.bs
!polyglot download sentiment2.ca
!polyglot download sentiment2.ce
!polyglot download sentiment2.ceb
!polyglot download sentiment2.cs
!polyglot download sentiment2.cv
!polyglot download sentiment2.cy
!polyglot download sentiment2.da
!polyglot download sentiment2.de
!polyglot download sentiment2.diq
!polyglot download sentiment2.dv
!polyglot download sentiment2.el
!polyglot download sentiment2.en
!polyglot download sentiment2.eo
!polyglot download sentiment2.es
!polyglot download sentiment2.et
!polyglot download sentiment2.eu
!polyglot download sentiment2.fa
!polyglot download sentiment2.fi
!polyglot download sentiment2.fo
!polyglot download sentiment2.fr
!polyglot download sentiment2.fy
!polyglot download sentiment2.ga
!polyglot download sentiment2.gan
!polyglot download sentiment2.gd
!polyglot download sentiment2.gl
!polyglot download sentiment2.gu
!polyglot download sentiment2.gv
!polyglot download sentiment2.he
!polyglot download sentiment2.hi
!polyglot download sentiment2.hif
!polyglot download sentiment2.hr
!polyglot download sentiment2.hsb
!polyglot download sentiment2.ht
!polyglot download sentiment2.hu
!polyglot download sentiment2.hy
!polyglot download sentiment2.ia
!polyglot download sentiment2.id
!polyglot download sentiment2.ilo
!polyglot download sentiment2.io
!polyglot download sentiment2.is
!polyglot download sentiment2.it
!polyglot download sentiment2.ja
!polyglot download sentiment2.jv
!polyglot download sentiment2.ka
!polyglot download sentiment2.kk
!polyglot download sentiment2.km
!polyglot download sentiment2.kn
!polyglot download sentiment2.ko
!polyglot download sentiment2.ku
!polyglot download sentiment2.ky
!polyglot download sentiment2.la
!polyglot download sentiment2.lb
!polyglot download sentiment2.li
!polyglot download sentiment2.lmo
!polyglot download sentiment2.lt
!polyglot download sentiment2.lv
!polyglot download sentiment2.mg
!polyglot download sentiment2.mk
!polyglot download sentiment2.ml
!polyglot download sentiment2.mn
!polyglot download sentiment2.mr
!polyglot download sentiment2.ms
!polyglot download sentiment2.mt
!polyglot download sentiment2.my
!polyglot download sentiment2.ne
!polyglot download sentiment2.nl
!polyglot download sentiment2.nn
!polyglot download sentiment2.no
!polyglot download sentiment2.oc
!polyglot download sentiment2.or
!polyglot download sentiment2.os
!polyglot download sentiment2.pa
!polyglot download sentiment2.pam
!polyglot download sentiment2.pl
!polyglot download sentiment2.pms
!polyglot download sentiment2.ps
!polyglot download sentiment2.pt
!polyglot download sentiment2.qu
!polyglot download sentiment2.rm
!polyglot download sentiment2.ro
!polyglot download sentiment2.ru
!polyglot download sentiment2.sa
!polyglot download sentiment2.sah
!polyglot download sentiment2.scn
!polyglot download sentiment2.sco
!polyglot download sentiment2.se
!polyglot download sentiment2.sh
!polyglot download sentiment2.si
!polyglot download sentiment2.sk
!polyglot download sentiment2.sl
!polyglot download sentiment2.sq
!polyglot download sentiment2.sr
!polyglot download sentiment2.su
!polyglot download sentiment2.sv
!polyglot download sentiment2.sw
!polyglot download sentiment2.szl
!polyglot download sentiment2.ta
!polyglot download sentiment2.te
!polyglot download sentiment2.tg
!polyglot download sentiment2.th
!polyglot download sentiment2.tk
!polyglot download sentiment2.tl
!polyglot download sentiment2.tr
!polyglot download sentiment2.tt
!polyglot download sentiment2.ug
!polyglot download sentiment2.uk
!polyglot download sentiment2.ur
!polyglot download sentiment2.uz
!polyglot download sentiment2.vec
!polyglot download sentiment2.vi
!polyglot download sentiment2.vls
!polyglot download sentiment2.vo
!polyglot download sentiment2.wa
!polyglot download sentiment2.war
!polyglot download sentiment2.yi
!polyglot download sentiment2.yo
!polyglot download sentiment2.zh
!polyglot download sentiment2.zhw

In [ ]:
# creating array of dates of the filenames that you will read in
dates_for_csv_filenames = []
for day in range(9, 21): 
    one_date = datetime.date(2023, 3, day)
    dates_for_csv_filenames.append(one_date)

print(dates_for_csv_filenames)

In [ ]:
def get_polarities(df):
    polarities = []
    for i in range(len(df)):
        # check for WARNING:polyglot.detect.base:Detector is not able to detect the language reliably
        # and assign sentiment of 999 to discard later because tweet will not be useful to analyze
        with warnings.catch_warnings(record=True) as w:
            one_tweet_content_text = Text(str(df.iloc[i].loc['content']))
            # strip text because sometimes there are spaces that are really just empty tweets after certain things have been removed
            # strip only strips the leading and trailing whitespace from strings
            one_tweet_content_text = one_tweet_content_text.strip()
            if w:
                polarities.append('999')

        # doesn't seem that any tweets get assigned sentiment == 333
        # think because a tweet of length 0 would have already triggered the warning and would have been assigned sentiment = 999
        # now it is since applying strip function to the text
        if len(one_tweet_content_text) == 0: 
            print('len(one_tweet_content_text) == 0')
            polarity = 333
        try:
            polarity = one_tweet_content_text.polarity
            detector = Detector(str(df.iloc[i].loc['content']))
        except ZeroDivisionError: 
            polarity = 0
        except Exception as e:
            polarity = 999

        polarities.append(polarity)

    # add column for original calculated polarities without factoring in # of likes or # of retweets 
    df['sentiment'] = polarities

    # cleaning data

    # dropping all rows with sentiment == 999 because mainly tweets with just numbers for content and other tweets not useful for analysis
    print(len(df))
    mask_999 = df['sentiment'] == 999
    df = df[~mask_999]
    print(len(df))

    mask_333 = df['sentiment'] == 333
    print(df[mask_333])
    print(len(df[mask_333]))

    num_content_nans = df['content'].isna().sum()
    print(f"num_content_nans: {num_content_nans}") 
    # drop rows with NaN values in the 'content' column
    df = df.dropna(subset=['content'])

    # create a column of just the dates not the time 
    just_dates = [df.iloc[i].loc['datetime'].split()[0] for i in range(len(df))]
    df['just_date'] = just_dates 

    return df

In [ ]:
def create_col_sentiment_likes_retweets_combo(df):
    # experimenting with factoring in the # of likes and # of retweets of a tweet into its sentiment score
    num_likes_normalized_plus_num_retweets_normalized = (df['num_likes_normalized']) + (df['num_retweets_normalized'])

    sentiment_num_likes_num_retweets = []
    # can mess with this # to weight how much the number of likes and number of retweets factors into the sentiment score, 
    weight_of_likes_retweets_in_sentiment_score = 10
    for i in range(len(df)): # maybe better pandas or numpy way to do this instead of with for loop
        if df.iloc[i].loc['sentiment'] > 0:
            sentiment_num_likes_num_retweets.append(df.iloc[i].loc['sentiment'] + (weight_of_likes_retweets_in_sentiment_score * num_likes_normalized_plus_num_retweets_normalized.iloc[i]))
        elif df.iloc[i].loc['sentiment'] < 0:
            sentiment_num_likes_num_retweets.append(df.iloc[i].loc['sentiment'] - (weight_of_likes_retweets_in_sentiment_score * num_likes_normalized_plus_num_retweets_normalized.iloc[i]))
        else:
            sentiment_num_likes_num_retweets.append(0)

    df['sentiment_num_likes_num_retweets'] = np.array(sentiment_num_likes_num_retweets)

    return df

In [ ]:
def get_day_from_file_and_preprocess(filename):
    one_file_df = pd.read_csv(one_day_filename)
    # drop 'Unnamed:' column is actually just unnamed and is the number index of the row in the csv file
    one_file_df.drop(one_file_df.columns[0], axis = 1, inplace=True)
    one_file_df = one_file_df.drop_duplicates(keep='first') # I don't believe there should be any duplicate tweets anymore, but just in case

    # check for nans in content (the tweet itself) and drop rows if they exist
    num_content_nans = one_file_df['content'].isna().sum()
    print(num_content_nans) 
    # drop rows with NaN values in the 'content' column
    one_file_df = one_file_df.dropna(subset=['content'])

    # getting just the dates without the times from datetime to be able to group by date
    just_dates = [one_file_df.iloc[i].loc['datetime'].split()[0] for i in range(len(one_file_df))]
    one_file_df['just_date'] = just_dates

    one_file_df = get_polarities(one_file_df)

    # normalize num_likes and num_retweets and add columns to the dataframe for each normalized version
    min_num_likes = one_file_df['num_likes'].min()
    max_num_likes = one_file_df['num_likes'].max()
    print(min_num_likes)
    print(max_num_likes)

    # normalizing the # of likes column
    one_file_df['num_likes_normalized'] = (one_file_df['num_likes'] - min_num_likes) / (max_num_likes - min_num_likes)

    min_num_retweets = one_file_df['num_retweets'].min()
    max_num_retweets = one_file_df['num_retweets'].max()
    print(min_num_retweets)
    print(max_num_retweets)

    # normalizing the # of retweets column
    one_file_df['num_retweets_normalized'] = (one_file_df['num_retweets'] - min_num_retweets) / (max_num_retweets - min_num_retweets)  

    # create column trying to combine original sentiment with # of likes and # of retweets in a useful way 
    one_file_df = create_col_sentiment_likes_retweets_combo(one_file_df)

    return one_file_df

In [ ]:
# read files into list of separate dataframes per date 
dataframes_by_day = []
for i in range(len(dates_for_csv_filenames)):
    one_day_filename = "bitcoin_just_" + dates_for_csv_filenames[i].strftime("%m"+"."+"%d"+"."+"%Y") + ".csv"
    print(one_day_filename)
    # doing preprocessing as well here to avoid looping back through
    one_file_df = get_day_from_file_and_preprocess(one_day_filename)
    dataframes_by_day.append(one_file_df)

In [ ]:
day_info_summaries = [] # array to hold all of the dictionaries
# dictionaries will contain key:value pairs 
# {KEY-DATE: VALUE-DATE, KEY-MEAN_WEIGHTED_SENTIMENT: VALUE-AVERAGE_SENTIMENT, KEY-NUM_TWEETS: VALUE-NUM_TWEETS}
for dataframe in dataframes_by_day:
    one_day_info_dict = {}
    one_day_info_dict['date'] = dataframe.iloc[0].loc['just_date']
    one_day_info_dict['mean_weighted_sentiment'] = dataframe['sentiment_num_likes_num_retweets'].mean()
    one_day_info_dict['num_tweets'] = len(dataframe)
    day_info_summaries.append(one_day_info_dict)

In [ ]:
print(day_info_summaries) # seems to work for getting the info that you want per day in dictionaries

In [ ]:
# seems to work out well, do for all of the days that you have saved
filename = 'summary_per_day.csv'
with open(filename, "w", newline="") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=day_info_summaries[0].keys())

    # write the header row one time 
    writer.writeheader()

    # then each dictionary without the header on the following lines
    for one_day_dict in day_info_summaries:
        writer.writerow(one_day_dict)